In [1]:
import sqlite3

import pandas as pd
from IPython.display import display

## Connect to SQLite database


In [2]:
# Define SQLite database file path
sqlite_db_path = "../data/vaccination_db.sqlite"

# Reconnect to SQLite database
conn = sqlite3.connect(sqlite_db_path)

## Query Database

To query the database, define the `query`. 

Next, execute the query and load the results into a DataFrame: `pd.read_sql_query(query, conn)`

### check info for all tables

In [16]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print(f"There are {len(tables)} tables in this sqlite dataset")
for table in tables:
    table_name = table[0]
    print(f"\nName of tables: {table_name}")

    # count rows for the table
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    row_count = cursor.fetchone()[0]
    print(f"row counts: {row_count}")
    # df = pd.read_sql(f"SELECT * FROM {table_name}", conn)
    # print(df.describe())

There are 6 tables in this sqlite dataset

Name of tables: Users
row counts: 200

Name of tables: Vaccines
row counts: 8

Name of tables: Polyclinics
row counts: 30

Name of tables: BookingSlots
row counts: 750

Name of tables: VaccineRecords
row counts: 500

Name of tables: VaccineStockInventory
row counts: 200


#### User

In [19]:
# Query the Vaccines table
query = "SELECT * FROM Users"
# Execute the query and load results into a DataFrame
user_df = pd.read_sql(query, conn)
# Display the results
display(user_df)

,user_id,name,email,date_of_birth,gender
0,1,Christina Edwards,cfloyd@example.net,1945-09-25,Female
1,2,Mark Medina,wilsonjorge@example.net,1980-09-20,Male
2,3,Tara Benjamin,jessica81@example.net,1971-11-25,Male
3,4,Angelica Hunt,kenneth60@example.org,1997-01-28,Female
4,5,Kristina Allen,sarah51@example.org,1974-10-20,Female
...,...,...,...,...,...
195,196,Brandy Clark,harold91@example.net,1963-10-14,Female
196,197,Daniel Cunningham,joann88@example.com,1947-01-29,Male
197,198,Kyle Hernandez,adkinsnatasha@example.net,1944-09-09,Female
198,199,Claudia Greene,danielpotter@example.com,1950-10-31,Female


In [ ]:
start_date = "1920-01-01"
end_date = "2025-03-01"

user_df["date_of_birth"] = pd.to_datetime(user_df["date_of_birth"])
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

is_in_range = (user_df["date_of_birth"] >= start_date) & (
    user_df["date_of_birth"] <= end_date
)

# check if the date is under range
if is_in_range.all():
    print("All date are in range")
else:
    print("Some dates are not in range")

All date are in range


#### Vaccines

In [3]:
# Query the Vaccines table
query = "SELECT * FROM Vaccines"
# Execute the query and load results into a DataFrame
vaccines_df = pd.read_sql(query, conn)
# Display the results
display(vaccines_df)

,vaccine_id,vaccine_name,vaccine_price,doses_required,age_criteria,gender_criteria
0,1,Influenza,$9,2,18+ years old,All
1,2,COVID-19,$0,4,18+ years old,All
2,3,Pneumococcal,$16,1,65+ years old,All
3,4,HIV,$23,2,18-26 years old,Female
4,5,Tdap,$10,1,27-64 years old,Female
5,6,HepB,$9,2,18+ years old,All
6,7,MMR,$9,1,18+ years old,All
7,8,Varicella,$11,3,18+ years old,All


#### Polyclinics

In [27]:
# Query the Vaccines table
query = "SELECT * FROM Polyclinics"
# Execute the query and load results into a DataFrame
polyclinic_df = pd.read_sql(query, conn)
# Display the results
display(polyclinic_df)

,polyclinic_id,polyclinic_name,address
0,1,"Riley, Myers and Bates Polyclinic","2888 Hoffman Ridge\nLake Jamieville, AR 37864"
1,2,Valdez-Thomas Polyclinic,"48790 Stanley Wells\nBrownmouth, AR 46951"
2,3,"Murphy, Salas and Boyle Polyclinic","5923 Steven Street\nSouth Rebecca, MT 88225"
3,4,Baxter and Sons Polyclinic,"650 Conner Unions Apt. 207\nDavishaven, MD 17473"
4,5,Warner Inc Polyclinic,77554 George Throughway Apt. 854\nWest Donaldv...
5,6,Hodges-Powell Polyclinic,"4884 Harrison Lakes\nPerrychester, PR 63590"
6,7,Martinez-Rogers Polyclinic,"153 Richard Lakes\nLake Matthewborough, AL 41603"
7,8,Maxwell-Schwartz Polyclinic,"6369 Berg Corner\nSouth Anne, NJ 75009"
8,9,Miller and Sons Polyclinic,"3998 Banks Parkway\nLeeport, MI 57146"
9,10,Snyder PLC Polyclinic,"420 Spencer Islands\nPetersonmouth, WV 07454"


#### BookingSlots

In [28]:
# Query the Vaccines table
query = "SELECT * FROM BookingSlots"
# Execute the query and load results into a DataFrame
bookingslot_df = pd.read_sql(query, conn)
# Display the results
display(bookingslot_df)

,slot_id,polyclinic_id,vaccine_id,slot_datetime,is_booked
0,1,29,2,2025-03-20 16:00:00,0
1,2,30,2,2025-03-19 09:00:00,0
2,3,4,1,2025-03-17 16:00:00,0
3,4,6,5,2025-03-21 11:00:00,0
4,5,5,1,2025-03-19 15:00:00,0
...,...,...,...,...,...
745,746,3,4,2025-03-18 10:00:00,1
746,747,2,8,2025-03-21 13:00:00,0
747,748,25,5,2025-03-17 12:00:00,0
748,749,8,2,2025-03-17 14:00:00,0


In [30]:
bookingslot_df["slot_datetime"].value_counts()

slot_datetime
2025-03-18 11:00:00    23
2025-03-21 10:00:00    23
2025-03-17 17:00:00    23
2025-03-20 11:00:00    21
2025-03-19 17:00:00    19
2025-03-21 11:00:00    19
2025-03-19 18:00:00    18
2025-03-20 18:00:00    18
2025-03-19 10:00:00    18
2025-03-20 09:00:00    17
2025-03-18 14:00:00    17
2025-03-20 15:00:00    17
2025-03-18 18:00:00    17
2025-03-18 17:00:00    17
2025-03-18 09:00:00    17
2025-03-21 16:00:00    17
2025-03-17 10:00:00    17
2025-03-21 13:00:00    17
2025-03-20 13:00:00    16
2025-03-21 18:00:00    16
2025-03-18 13:00:00    16
2025-03-20 10:00:00    15
2025-03-21 14:00:00    15
2025-03-19 14:00:00    15
2025-03-18 10:00:00    15
2025-03-17 18:00:00    15
2025-03-18 16:00:00    15
2025-03-19 16:00:00    15
2025-03-19 15:00:00    15
2025-03-17 16:00:00    14
2025-03-20 16:00:00    14
2025-03-17 15:00:00    14
2025-03-17 09:00:00    14
2025-03-21 17:00:00    14
2025-03-18 15:00:00    14
2025-03-21 15:00:00    13
2025-03-20 14:00:00    13
2025-03-19 13:00:00    1

#### VaccineRecords

In [31]:
# Query the Vaccines table
query = "SELECT * FROM VaccineRecords"
# Execute the query and load results into a DataFrame
vaccinerecord_df = pd.read_sql(query, conn)
# Display the results
display(vaccinerecord_df)

,record_id,user_id,vaccine_id,polyclinic_id,vaccination_date
0,1,115,6,2,2025-01-18
1,2,33,2,4,2025-01-21
2,3,121,5,10,2025-02-10
3,4,135,2,23,2025-01-21
4,5,166,4,7,2025-03-15
...,...,...,...,...,...
495,496,152,7,14,2025-02-22
496,497,54,4,21,2025-01-23
497,498,47,8,25,2025-02-24
498,499,64,5,29,2025-03-04


In [34]:
vaccinerecord_df["vaccination_date"].sort_values()

437    2025-01-01
329    2025-01-01
479    2025-01-01
372    2025-01-01
288    2025-01-01
          ...    
421    2025-03-15
276    2025-03-15
444    2025-03-15
398    2025-03-16
312    2025-03-16
Name: vaccination_date, Length: 500, dtype: object

#### VaccineStockInventory

In [35]:
# Query the Vaccines table
query = "SELECT * FROM VaccineStockInventory"
# Execute the query and load results into a DataFrame
vaccinestock_df = pd.read_sql(query, conn)
# Display the results
display(vaccinestock_df)

,inventory_id,polyclinic_id,vaccine_id,stock_quantity
0,1,18,7,55
1,2,18,3,183
2,3,18,4,79
3,4,18,1,92
4,5,11,7,151
...,...,...,...,...
195,196,6,1,175
196,197,18,2,96
197,198,18,1,84
198,199,18,3,192


## Close SQLite Database Connection


In [36]:
# Close the connection
conn.close()